In [16]:
import os
import re
import numpy as np
from PIL import Image
from random import shuffle

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [18]:
np.random.seed(1337)  # for reproducibility

In [19]:
batch_size = 200
nb_classes = 2
nb_epoch = 25

In [20]:
# input image dimensions
img_rows, img_cols = 100, 100

# input shape
input_shape = (img_rows, img_cols, 1)

In [13]:
def img_to_matrix(filename):
    """
    takes a filename and turns it into a numpy array of greyscale pixels
    """
    img = Image.open(filename).convert('L')
    img = img.resize((img_rows, img_cols))
    img = list(img.getdata())
    img = np.array(img)
    img = img.reshape(img_rows, img_cols, 1)
    return img

In [14]:
img_dir = "train/"
image_files = [img_dir + f for f in os.listdir(img_dir)]

shuffle(image_files)

train_size = 5 * len(image_files) // 6

train_images = image_files[:train_size]
test_images = image_files[train_size:]

In [21]:
X_train = [img_to_matrix(train_image) for train_image in train_images]
Y_train = [0 if "cat" in f.split('/')[-1] else 1 for f in train_images]

X_test = [img_to_matrix(test_image) for test_image in test_images]
Y_test = [0 if "cat" in f.split('/')[-1] else 1 for f in test_images]

X_train = np.array(X_train)
X_test = np.array(X_test)

In [22]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [23]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (20833, 100, 100, 1)
20833 train samples
4167 test samples


In [24]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

In [25]:
model = Sequential()

In [26]:
model.add(Convolution2D(8, 9, 9, input_shape=input_shape))
model.add(Activation('relu'))

In [27]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [28]:
model.add(Convolution2D(16, 7, 7))
model.add(Activation('relu'))

In [29]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [30]:
model.add(Convolution2D(32, 5, 5))
model.add(Activation('relu'))

In [31]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [32]:
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))

In [33]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [34]:
model.add(Convolution2D(250, 3, 3))
model.add(Activation('relu'))

In [35]:
model.add(Flatten())
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [36]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [37]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 20833 samples, validate on 4167 samples
Epoch 1/25
20833/20833 [==============================] - 28s - loss: 0.6912 - acc: 0.5305 - val_loss: 0.6868 - val_acc: 0.5491
Epoch 2/25
20833/20833 [==============================] - 24s - loss: 0.6822 - acc: 0.5630 - val_loss: 0.7251 - val_acc: 0.4994
Epoch 3/25
20833/20833 [==============================] - 24s - loss: 0.6724 - acc: 0.5810 - val_loss: 0.7170 - val_acc: 0.5404
Epoch 4/25
20833/20833 [==============================] - 24s - loss: 0.6519 - acc: 0.6184 - val_loss: 0.6392 - val_acc: 0.6338
Epoch 5/25
20833/20833 [==============================] - 24s - loss: 0.6136 - acc: 0.6660 - val_loss: 0.5824 - val_acc: 0.6935
Epoch 6/25
20833/20833 [==============================] - 24s - loss: 0.5727 - acc: 0.7012 - val_loss: 0.6566 - val_acc: 0.6367
Epoch 7/25
20833/20833 [==============================] - 24s - loss: 0.5360 - acc: 0.7289 - val_loss: 0.5475 - val_acc: 0.7183
Epoch 8/25
20833/20833 [==============================]

In [38]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.559454110348
Test accuracy: 0.772738180974


In [40]:
# Save model
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")